# Collect data for GitHub Action checks/workflow runs

In this notebook, we collect historical test data like the test duration values from running workflows on Github using the GitHub API.

From historical test workflow runs, we want to extract:

- time durations
- workflow run status & conclusion

We can get workflow IDs of the test that we are interested in from https://api.github.com/repos/{ORG}/{REPO}/actions/workflows 

In [1]:
# import libraries
from dotenv import find_dotenv, load_dotenv
import os
import json
import subprocess
from datetime import datetime
from subprocess import PIPE
import pandas as pd
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

import warnings

warnings.filterwarnings("ignore")

## Helper Functions

In [5]:
def get_page_numbers(test_id):
    """
    Get the total count of tests.
    Find the pages on github-actions.
    """
    command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?""".format(TOKEN, ORG, REPO, test_id)
    args = []
    args.append(command)
    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)
    total_count = output['total_count']
    page_numbers = int(total_count/30) # by default number of tests on one page is 30
    return page_numbers

In [6]:
def get_workflow_runs(test_id, page_numbers):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    for p in range(1,page_numbers+1):
        command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?page={}""".format(TOKEN, ORG, REPO, test_id, p)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        if p==1:
            df = pd.json_normalize(output['workflow_runs'])
        else:
            df2 = pd.json_normalize(output['workflow_runs'])
            df = pd.concat([df, df2], axis=0)
    return df

In [7]:
def get_check_runs(commits):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    appended_data = []
    for commit in commits:
        command = """curl -L \
          -H "Accept: application/vnd.github+json" \
          -H "Authorization: Bearer {}"\
          -H "X-GitHub-Api-Version: 2022-11-28" \
          https://api.github.com/repos/{}/{}/commits/{}/check-runs""".format(TOKEN, ORG, REPO, commit)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        appended_data.append(pd.json_normalize(output['check_runs']))

    df = pd.concat(appended_data, axis=0)

    return df

In [8]:
def get_commits():
    "This function gets all commit refs for a github repo"

    command =  """curl -L \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/commits""".format(TOKEN, ORG, REPO)
    args = []
    args.append(command)

    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)

    df = pd.json_normalize(output)
    return df

## Data Collection

We will be fetching the GitHub Action workflows and checks for the repo: https://www.github.com/github/codeql.

In [2]:
load_dotenv(find_dotenv(), override=True)
TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
ORG = os.getenv("ORG")
REPO = os.getenv("REPO")

In [4]:
# mode can be "check or "workflow" depending on whether you want to collect checks data or workflows data
MODE = "workflow"
test_id = "14909022" # "CodeQL tests"

* Workflows: For example, lets collect data for the workflow ID 14909022 for the workflow runs in the repository `github/codeql` https://api.github.com/repos/github/codeql/actions/workflows

* Checks: Get commits for a repo and get check runs for each commit

In [9]:
if MODE == "check":
    # get checks
    df_commits = get_commits()
    commit_ids = list(df_commits['sha'])
    checks_df = get_check_runs(commit_ids)
    test_df = checks_df[['started_at', 'completed_at', 'id', 'status', 'conclusion', 'external_id', 'name']]
    test_df = test_df.rename(columns={'external_id': 'test_id', 'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['completed_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['started_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)

elif MODE == "workflow":
    # get workflows
    page_numbers = get_page_numbers(test_id)
    workflow_df = get_workflow_runs(test_id, page_numbers)
    test_df = workflow_df[['created_at', 'updated_at', 'id', 'status', 'conclusion']]
    test_df = test_df.rename(columns={'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['updated_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['created_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)
    test_df['test_id'] = test_id

In [10]:
test_df.head(10)

,created_at,updated_at,run_id,status,conclusion,run_duration,test_id
0,2023-06-06T10:10:02Z,2023-06-06T10:10:16Z,5187296010,completed,success,14.0,14909022
1,2023-06-06T08:56:19Z,2023-06-06T08:56:40Z,5186591599,completed,success,21.0,14909022
2,2023-06-05T17:07:18Z,2023-06-05T17:07:31Z,5179907330,completed,success,13.0,14909022
3,2023-06-05T08:59:42Z,2023-06-05T08:59:57Z,5175246940,completed,success,15.0,14909022
4,2023-06-02T13:58:51Z,2023-06-02T13:59:04Z,5156289485,completed,success,13.0,14909022
5,2023-06-02T10:52:07Z,2023-06-02T10:52:22Z,5154706775,completed,success,15.0,14909022
6,2023-06-02T10:51:14Z,2023-06-02T10:51:28Z,5154700372,completed,failure,14.0,14909022
7,2023-06-01T22:06:34Z,2023-06-01T22:06:48Z,5149673307,completed,success,14.0,14909022
8,2023-06-01T15:25:54Z,2023-06-01T15:26:10Z,5146241506,completed,success,16.0,14909022
9,2023-05-31T16:05:40Z,2023-05-31T16:05:57Z,5135136494,completed,success,17.0,14909022


In [11]:
test_df.shape

(1020, 7)

In [12]:
# generating passing and failing dfs which are neccesary for computing fit distributions
passing_df = test_df[test_df['conclusion'] == 'success'] 
failures_df = test_df[test_df['conclusion'] == 'failure'] 

In [13]:
passing_df.shape

(964, 7)

In [14]:
failures_df.shape

(56, 7)

In [15]:
# Split the data into train and test.
passing_train, passing_test = train_test_split(passing_df, test_size=0.2, random_state=25)

In [16]:
# Split the data into train and test
failing_train, failing_test = train_test_split(failures_df, test_size=0.2, random_state=25)

In [17]:
passing_train.shape

(771, 7)

In [18]:
passing_test.shape

(193, 7)

In [19]:
failing_train.shape

(44, 7)

In [20]:
failing_test.shape

(12, 7)

In [22]:
passing_train.to_csv("../data/processed/{}passing_train.csv".format(test_id))
failing_train.to_csv("../data/processed/{}failing_train.csv".format(test_id))
passing_test.to_csv("../data/processed/{}passing_test.csv".format(test_id))
failing_test.to_csv("../data/processed/{}failing_test.csv".format(test_id))

## Conclusion

In this notebook, we interact with the github api to collect the data for all workflow runs. In future work, we will look into using this data to perform statistical tests using OSP model.